## DeepUnitMatch training notebook

This demo notebook will take you through training DeepUnitMatch on your own data. This is not necessary, as the neural network is able to generalise to unseen datasets, but training on your own data is very likely to improve performance.       


In [ ]:
import torch
from DeepUnitMatch.train import train_AE, train_finetune
from DeepUnitMatch.utils import train_utils, AE_npdataset, npdataset, mymodel

### Data loading

In order to easily load your own Neuropixels 2.0 data into the neural network for training, there are custom pytorch datasets. These load data with a hierarchical structure:

[ROOT]/[MOUSE]/[PROBE]/[LOCATION]/[EXPERIMENT]/processed_waveforms

Each 'processed_waveforms' directory should contain the .npy files, one per waveform, each of shape (60, 30, 2) -- (timepoints, channels, session halves) 

Assuming you have a decent amount of data in this format, we can perform the two training steps of DeepUnitMatch